In [1396]:
import json
#from functools import partial
import functools
from inspect import getsource
from math import log
from sys import argv
from bisect import bisect_left
import re
import gzip
import zlib
def checker(nm,ori):
    flag = False
    for key in ori.keys():
        if ori[key] != nm[key]:
            flag = True
            break
    print('Match!!' if flag == False else 'Error!!')

def test(origin_mapping,idx):
    print(len(idx))
    mark = consecu_int_run_length(idx)
    # check part
    _m = mark
    nm = {}
    for i in range(len(keys)):
        nm[keys[i]] = values[i]
    for i in range(0,len(_m),3):
        for j in range(_m[i+2]):
            nm[keys[_m[i+1]+j]] = values[_m[i]+j]
    checker(nm,origin_mapping)

    sporadic_seg = sporadic_seg_run_length(mark)
    # check part
    _m = sporadic_seg
    nm = {}
    vv = set(values)
    for i in range(len(keys)):
        if keys[i] in values:
            nm[keys[i]] = keys[i]
        #nm[keys[i]] = values[i]
    for i in range(0,len(_m),3):
        if _m[i+2] != 1:
            for j in range(_m[i+2]):
                nm[keys[_m[i+1]+j]] = values[_m[i]+j]
        else:
            length = len(_m[i+1])
            for j in range(length):
                nm[keys[_m[i+1][j]]] = values[_m[i]+j]
    checker(nm,origin_mapping)

def consecu_int_run_length(idx):
    mark = []
    flag = False
    temp = []
    for i in range(len(idx)):
        if i != idx[i]:      
            if flag == False:
                temp = [ i, idx[i], 1 ]
                flag = True
            elif flag == True and idx[i]-(temp[1]+temp[2]) == 0:
                temp[2] += 1
            elif flag == True:
                mark += temp
                temp = [ i, idx[i], 1 ]
                flag = True
        else:
            if flag == True:
                mark += temp
                temp = []
                flag = False
    if len(temp) != 0:
        mark += temp
    return mark

def sporadic_seg_run_length(mark):
    sporadic_seg = [ mark[i:i+3] for i in range(0,len(mark),3)]
    flag = False
    temp = []
    res = []
    for i in range(len(sporadic_seg)):
        if sporadic_seg[i][2] != 1:
            if flag == True:
                res += temp 
                # initialize temp to an empty list, to avoid if temp have something but index is at the end
                temp = []
                flag = False
            res += sporadic_seg[i]
        else:
            # first one
            if flag == False:
                # start index, index list, len
                temp = [sporadic_seg[i][0],[sporadic_seg[i][1]],1]
                flag = True
            else:
                if sporadic_seg[i][0] - sporadic_seg[i-1][0] == 1:
                    temp[1].append(sporadic_seg[i][1]) 
                else:
                    res += temp  
                    temp = [sporadic_seg[i][0],[sporadic_seg[i][1]],1]
    if len(temp) != 0:
        res += temp  
    return res

def classify(origin_mapping):
    same, prefix, infix, postfix, others = [], [], [], [], []
    for k,v in origin_mapping.items():
        index = v.find(k)
        if k == v:
            same += [k]
        elif index != -1:
            if index == 0:
                prefix += [k]
            elif v[index:] == k:
                postfix += [k]
            else:
                infix += [k]
        else:
            others += [k]
    print('same: {}\nprefix: {}\ninfix: {}\npostfix: {}\nothers: {}'.format(len(same),len(prefix),len(infix),len(postfix),len(others)))
    return same, prefix, infix, postfix, others
g=len
def p(x,y):
    d,b=g(x[0]),g(y[0])
    if x[1]==y[1]:
        if d-b:
            return 1 if d>b else -1
        else:
            for i in range(d):
                if x[0][i]==y[0][i]:
                    continue
                return 1 if int(x[0][i])>int(y[0][i]) else -1       
            return 0
    return 1 if x[1]>y[1] else -1
def t(l):
    r,e=[],[]
    for i in range(g(l)):
        y = re.findall('([0-9]+)',l[i])
        if g(y) != 0:
            a = list(re.finditer('([0-9]+)',l[i]))
            c=l[i][:]
            for j in range(len(a)):
                c=c.replace(a[j].group(0),'',1)
            r.append(c)
        else:
            r.append(l[i])
        e.append(y)
    return e,r
    
document3 = """import json as a,sys,re
from functools import cmp_to_key
g=len
s=sorted
def p(x,y):
    d,b=g(x[0]),g(y[0])
    if x[1]==y[1]:
        if d-b:
            return 1 if d>b else -1
        else:
            for i in range(d):
                if x[0][i]==y[0][i]:
                    continue
                return 1 if int(x[0][i])>int(y[0][i]) else -1       
            return 0
    return 1 if x[1]>y[1] else -1
def t(l):
    r,e=[],[]
    for i in range(g(l)):
        y=re.findall('([0-9]+)',l[i])
        if g(y) != 0:
            d = list(re.finditer('([0-9]+)',l[i]))
            c=l[i][:]
            for j in range(len(d)):
                c=c.replace(d[j].group(0),'',1)
            r.append(c)
        else:
            r.append(l[i])
        e.append(y)
    return e,r
n,o=sys.argv[1:3]
k,v=a.load(open(n))
w,x=t(k)
y,z=t(v)
c=s(zip(w,x,k),key=cmp_to_key(p))
b=s(zip(y,z,v),key=cmp_to_key(p))
a.dump(dict([(c[i][2],b[i][2]) for i in range(g(c))]),open(o,'w'))"""

def extract_remove(lst):
    remove, extract = [], []
    for i in range(len(lst)):
        integers = re.findall('([0-9]+)',lst[i])
        if len(integers) != 0:
            integer_lst = list(re.finditer('([0-9]+)',lst[i]))
            cpy = lst[i][:]
            for j in range(len(integer_lst)):
                cpy = cpy.replace(integer_lst[j].group(0),'',1)
            remove.append(cpy)
        else:
            remove.append(lst[i])
        extract.append(integers)
    return extract, remove

def bit_size(lst):
    maximum = 0
    for item in lst:
        all_int = re.findall('\d+',item)
        if not all_int:
            continue
        maximum = max(maximum, (len(all_int[0]) if len(all_int) == 1 else max([*map(len,all_int)])))        
    return maximum

def zfill_method(origin_mapping,maximum):
    # mmm=[k:('','',...),v:('','',...)] 
    ori=[*zip(*origin_mapping.items())]

    # ori_zfill =[0:['','',...],1:['','',...]]
    ori_zfill=[*map(lambda x:[*map(lambda y:re.sub(r'(\d+)',lambda z:z.group(1).zfill(maximum),y),x)],ori)]

    # ori_and_zfill = [[(k1,kf1),(k2,kf2),...],[(v1,vf1),(v2,vf2),...]]
    ori_and_zfill=[*map(lambda x,y:[*zip(x,y)],ori,ori_zfill)]

    # sort_by_zfill = [[(k1,kf1),(k2,kf2),...],[(v1,vf1),(v2,vf2),...]]
    sort_by_zfill=[*map(lambda x:sorted(x,key=lambda y:y[1]),ori_and_zfill)]

    # transpose=[0:[(k1,k2,...),(kf1,kf2,...)],1:[(v1,v2,...),(vf1,vf2,...)]] 
    transpose = [*map(lambda x: [*zip(*x)],sort_by_zfill)]
    new_table=dict(zip(transpose[0][0],transpose[1][0]))
    for k,v in origin_mapping.items():
        if new_table[k] != v:
            return False
    return True

def check_type(origin_mapping):
    # check largest bit size
    maximum = max(bit_size(origin_mapping.keys()),bit_size(origin_mapping.values()))
    print('bit_size:',maximum)
    same, prefix, infix, postfix, others = classify(origin_mapping)
    
    #CASE0
    flag = 0
    keys, values = zip(*origin_mapping.items())
    keys, values = sorted(keys), sorted(values)
    for i in range(len(keys)):
        if values[i] != origin_mapping[keys[i]]:
            flag = 1
            break
    if flag == 0:
        return 0
    
    # zfill_method
    if zfill_method(origin_mapping,maximum):
        return 1
    
#     #EXTRACT METHOD
#     #kv_lst = sorted(list(origin_mapping.items()),key=lambda x: x[0])
#     kv_lst = list(origin_mapping.items())
#     # divide kv_list to k and v
#     k,v = zip(*kv_lst)
#     k = list(k)
#     v = list(v)   
#     # ke: extraction of int of key
#     # ve: extraction of int of value
#     # kr: string removed int part of key
#     # vr: string removed int part of value
#     ke, kr = t(k)
#     ve, vr = t(v)
#     table = zip(ke,kr,ve,vr,k,v)
#     table = sorted(table,key=cmp_to_key(p))      
#     t0,t1,t2,t3,t4,t5 = zip(*table)
#     check_table = zip(t2,t3,[i for i in range(len(k))])
#     check_table = sorted(check_table,key=cmp_to_key(p))
#     ccc = 0
#     for i in range(len(k)):
#         if check_table[i][2] != i:
#             ccc += 1
#             #print(table[i-2:i+1])
#             #break
#     print('shift: {}'.format(ccc))
#     if ccc == 0:
#         return 2
    
    
    print('using my own comparator')
    i0,i1,i2=zip(*check_table)
    #print('original idx list char count: {}'.format(len(str(list(i2)))))
    i2temp = consecu_int_run_length(list(i2))
    print('consecu idx list char count: {}'.format(len(str(i2temp))))
    i2temp1 = sporadic_seg_run_length(i2temp)
    print('sporadic_seg idx list char count: {}'.format(len(str(i2temp1))))
    print()
    
    print('using default comparator')
    mmm = sorted(list(origin_mapping.items()),key=lambda x: x[0])
    
    k,v = zip(*mmm)
    k = list(k)
    v = list(v)          
    mm = sorted(zip(v,[ i for i in range(len(k)) ]),key=lambda x: x[0])
    v, idx = zip(*mm) 
    idx = list(idx)
    mark = consecu_int_run_length(idx)
    print('consecu idx list char count: {}'.format(len(str(mark))))
    sporadic_seg = sporadic_seg_run_length(mark)
    print('sporadic_seg idx list char count: {}'.format(len(str(sporadic_seg))))
    print()
    return 2


def A(rhs,lhs):
    return 1 if int(rhs)>int(lhs) else -1
if __name__ == '__main__':
    case =4
    print('case: {}'.format(case))
    map_in_name, script_name = '../cases/case{}.json'.format(case), 'test.py'
    nf = '../input/case{}_in.json'.format(case)
    nf = open(nf, 'r')
    nf = json.load(nf)
    keys, values = sorted(nf[0]), sorted(nf[1])
    
    map_in_file = open(map_in_name)
    origin_mapping = json.load(map_in_file)
    map_in_file.close()
   
    print('type: {}'.format(check_type(origin_mapping)))
    
       
    
    '''
    v, idx = zip(*mm) 
    
    idx = list(idx)
    v = list(v)
    mark = consecu_int_run_length(idx)
    print('len of str of mark: {}'.format(len(str(mark))))
    sporadic_seg = sporadic_seg_run_length(mark)
    print('len of str of sporadic_seg: {}'.format(len(str(sporadic_seg))))'''
    document1 = """import json as a,sys 
w={}
n,o=sys.argv[1:3]
k,v=a.load(open(n))
s=sorted
k,v=set(k),set(v)
r=k&v
c=list(r)
m=dict(zip(c,c))
k,v=s(k-r),s(v-r)
l=len
g=range
for i in g(l(k)):
    m[k[i]]=v[i]
for i in g(0,l(w),3):
    if w[i+2]!=1:
        for j in g(w[i+2]):
             m[k[w[i+1]+j]]=v[w[i]+j]
    else:
        for j in g(l(w[i+1])):
            m[k[w[i+1][j]]]=v[w[i]+j]
a.dump(m,open(o,'w'))""".format(str(sporadic_seg))

    document2 = b"""import json as a,sys
n,o=sys.argv[1:3]
k,v=a.load(open(n))
s=sorted
a.dump(dict(zip(s(k),s(v))),open(o,'w'))"""   
    document4 = """import json as a,sys
n,o=sys.argv[1:3]
A=a.load(open(n))
B=[*map(lambda x:[*zip(*x)],[*map(lambda x:sorted(x,key=lambda y:y[1]),[*map(lambda m,n:[*zip(m,n)],A,[*map(lambda x:[*map(lambda y:re.sub(r'(\d+)',lambda z:z.group(1).zfill({}),y),x)],A)])])])]
a.dump(dict(zip(B[0][0],B[1][0])),open(o,'w'))""".format(max(bit_size(origin_mapping.keys()),bit_size(origin_mapping.values())))

    #script=open(script_name,'w')
    #script.write(document1 if len(sporadic_seg) != 0 else document2)
    #script.close()

case: 4
bit_size: 4
same: 11587
prefix: 1046
infix: 15
postfix: 0
others: 21083
type: 1


[[1, 2, 3], [1, 2, 3], [1, 2, 3]]


In [1149]:
string3 = 'INCROSS_LAG_VIPS/TIU_FIGS_tob_sai[0][0]_DR_tob_sai[0][1]_DR_tob_sai[0][2]_DR_tob_sai[0][3]/unta_$O1/lao1/O$1'
string5 = 'INCROSS_LAG_VIPS/TIU_FIGS_tob_sai_DR_tob_sai_DR_tob_sai_DR_tob_sai/unta_$O1/lao1/O$1'
string4 = 'INCROSS_LAG_VIPS/TIU_FIGS_tob_sai[0][0]_DR_tob_sai[0][1]/unta_$O1/lao1/O$1'
string6 = 'INCROSS_LAG_VIPS/TIU_FIGS_tob_sai[5][8]_DR_tob_sai[5][15]/unta_$O1/lao1/O$1'

d='[0-9]+' 
a=lambda x:re.findall(d,x)
c=lambda x:re.sub(r'(\d+)',lambda m:m.group(1).zfill(4),x)
#c=lambda x:''.join(sum(([*i]for i in zip(re.split('(?:\d+)',x),[*map(lambda y:y.zfill(4),re.findall('(\d+)',x))])),[])) if re.findall('(\d+)',x) else x
Q=lambda x,y: 1 if c(x[0])>c(y[0])else -1

fff='''import json as _,sys,re
from functools import cmp_to_key
d='[0-9]+' 
a=lambda x:re.findall(d,x)
c=lambda x:re.sub(d,'',x)+str(len(a(x)))+''.join([*map(lambda y:y.zfill(4),a(x))])
n,o=sys.argv[1:3]
_.dump(dict(zip(*map(lambda t:sorted(list(t),key=cmp_to_key(lambda x,y: 1 if c(x[0])>c(y[0])else -1)),zip(*a.load(open(n)))))),open(o,'w'))'''

print(len(fff))
#mmm = [ it for it in origin_mapping.items() if it[0] not in same ]
print(len(origin_mapping),len(mmm),len(same))
mmm = list(origin_mapping.items())
mmm=sorted(mmm,key=cmp_to_key(Q))
a0,a1 =zip(*mmm)
aaa = list(zip(a1,[i for i in range(len(mmm))]))

aaa=sorted(aaa,key=cmp_to_key(Q))
ccc = 0
for i in range(len(mmm)):
    if i != aaa[i][1]:
        print('start:',i)
        break
        ccc += 1
v, idx = zip(*aaa) 
idx = list(idx)

for i in range(1459,1569,1):
    print(a0[i],aaa[i][0],aaa[i][1])


print('shift count: %d' % ccc)
mark = consecu_int_run_length(idx)
print('consecu idx list char count: {}'.format(len(str(mark))))
sporadic_seg = sporadic_seg_run_length(mark)
print('sporadic_seg idx list char count: {}'.format(len(str(sporadic_seg))))
print()
#print(extract_append(string6,string3))

336
62726 62726 3073
dhow_ad_ungt[22] dhow_ad_ungt[22] 1459
dhow_ad_ungt[23] dhow_ad_ungt[23] 1460
dhow_ad_ungt[24] dhow_ad_ungt[24] 1461
dhow_ad_ungt[25] dhow_ad_ungt[25] 1462
dhow_ad_ungt[26] dhow_ad_ungt[26] 1463
dhow_ad_ungt[27] dhow_ad_ungt[27] 1464
dhow_ad_ungt[28] dhow_ad_ungt[28] 1465
dhow_ad_ungt[29] dhow_ad_ungt[29] 1466
dhow_ad_ungt[30] dhow_ad_ungt[30] 1467
dhow_croy_sod dhow_croy_sod 1468
dhow_hardtack2_anaunters/anaunters_ag_gens_ket_fish/s_qid dhow_hardtack2_anaunters/anaunters_ag_gens_ket_fish/s_qid/frey3 1469
dhow_hardtack2_anaunters/anaunters_anti_beslow_qid/s_qid[0] dhow_hardtack2_anaunters/anaunters_anti_beslow_qid/s_qid[0]/frey2 1470
dhow_hardtack2_anaunters/anaunters_ill_qid/s_qid dhow_hardtack2_anaunters/anaunters_ill_qid/s_qid/frey2 1471
dhow_hardtack2_anaunters/defi_oxy_jah_asa_qid/s_qid dhow_hardtack2_anaunters/defi_oxy_jah_asa_qid/s_qid/frey2 1472
dhow_hardtack2_anaunters/defi_oxy_luxe_gray_qid/s_qid dhow_hardtack2_anaunters/defi_oxy_luxe_gray_qid/s_qid/frey2

In [909]:
string3 = 'INCROSS_LAG_VIPS/TIU_FIGS_tob_sai[0][0]_DR_tob_sai[0][1]_DR_tob_sai[0][2]_DR_tob_sai[0][3]/unta_$O1/lao1/O$1'
string4 = 'INCROSS_LAG_VIPS/TIU_FIGS_tob_sai[5][8]_DR_tob_sai[5][15]/unta_$O1/lao1/O$1'
string5 = 'INCROSS_LAG_VIPS/TIU_FIGS_tob'
#B:dimensions(string,i)
#C:order(string)
#D:dictionary_order(rhs,lhs)
#E:iter_bit(dim,rhs,lhs)
#F:dimension_based(rhs,lhs)
#S:string
#R:rhs
#L:lhs
def B(S,i):
    it=list(re.finditer('{}'.format('\[([0-9]+)\]'*i),S))
    return(0,0)if not i else(i,it)if len(it)else B(S,i-1)
def C(S):
    it=re.findall('\$O([0-9]+)',S)
    return int(it[0]) if len(it) else 0
def D(R,L):
    return 1 if R>L else 0 if R==L else -1
def E(I,R,L):
    for i in range(I):
        if R[i]!=L[i]:
            return A(R[i],L[i])
    return 0
def F(R,L):
    M=B(R[0],2)
    N=B(L[0],2)
    if M[0]==N[0] and M[0]!=0:
        X,Y=C(R[0]),C(L[0])
        if R[0][:M[1][0].start()]==L[0][:N[1][0].start()]:
            Z=(1 if X>Y else -1)if X-1>=len(M[1])and Y-1>=len(N[1])else(1 if X-1>=len(M[1])else -1)if X-1>=len(M[1])or Y-1>=len(N[1])else E(M[0],M[1][X-1].groups(),N[1][Y-1].groups())
            return Z if Z else 1 if X>Y else-1 
    return D(R[0],L[0])

mmm = list(origin_mapping.items())
mmm=sorted(mmm,key=cmp_to_key(F))
a0,a1 =zip(*mmm)
aaa = list(zip(a1,[i for i in range(len(mmm))]))
aaa=sorted(aaa,key=cmp_to_key(F))
ccc = 0
for i in range(len(mmm)):
    if i != aaa[i][1]:
        ccc += 1
print(ccc)
for i in range(8250,8350,1):
    if origin_mapping[a0[i]] == aaa[i][0]:
        print('A:',a0[i])
        print('B:',aaa[i][0])
    else:
        print('A:',a0[i])
        print('B:',aaa[i][0])
        print('*B should be',origin_mapping[a0[i]])
    
v, idx = zip(*aaa) 
idx = list(idx)
mark = consecu_int_run_length(idx)
print('consecu idx list char count: {}'.format(len(str(mark))))
sporadic_seg = sporadic_seg_run_length(mark)
print('sporadic_seg idx list char count: {}'.format(len(str(sporadic_seg))))
print()

abc = '''def A(R,L):
    return 1 if int(R)>int(L) else -1
def B(S,i):
    it=list(re.finditer('{}'.format('\[([0-9]+)\]'*i),S))
    return(0,0)if not i else(i,it)if len(it)else B(S,i-1)
def C(S):
    it=re.findall('\$O([0-9]+)',S)
    return int(it[0]) if len(it) else 0
def D(R,L):
    return 1 if R>L else 0 if R==L else -1
def E(I,R,L):
    for i in range(I):
        if R[i]!=L[i]:
            return A(R[i],L[i])
    return 0
def F(R,L):
    M=B(R[0],2)
    N=B(L[0],2)
    if M[0]==N[0] and M[0]!=0:
        X,Y=C(R[0]),C(L[0])
        if R[0][:M[1][0].start()]==L[0][:N[1][0].start()]:
            Z=(1 if X>Y else -1)if X-1>=len(M[1])and Y-1>=len(N[1])else(1 if X-1>=len(M[1])else -1)if X-1>=len(M[1])or Y-1>=len(N[1])else E(M[0],M[1][X-1].groups(),N[1][Y-1].groups())
            return Z if Z else 1 if X>Y else-1 
    return D(R[0],L[0])'''
qqq = '''def A(R,L):
    return 1 if int(R)>int(L) else -1
def B(S,i):
    it=list(re.finditer('{}'.format('\[([0-9]+)\]'*i),S))
    return (0,0) if not i else (i,it) if len(it) else B(S,i-1)
def C(S):
    it=re.findall('\$O([0-9]+)',S)
    return int(it[0]) if len(it) else 0
def D(R,L):
    return 1 if R>L else 0 if R==L else -1
def E(dim,R,L):
    for i in range(dim):
        if R[i]!=L[i]:
            return A(R[i],L[i])
    return 0
def F(R,L):
    rd=B(R[0],2)
    ld=B(L[0],2)
    if rd[0]==ld[0] and rd[0]!=0:
        X,Y=C(R[0]),C(L[0])
        if R[0][:rd[1][0].start()]==L[0][:ld[1][0].start()]:
            if X-1>=len(rd[1]) and Y-1>=len(ld[1]):
                return 1 if X>Y else -1
            if X-1>=len(rd[1]) or Y-1>=len(ld[1]):
                return 1 if X-1>=len(rd[1]) else -1
            res=E(rd[0],rd[1][X-1].groups(),ld[1][Y-1].groups())
            return res if res else 1 if X > Y else -1 
    return D(R[0],L[0])'''
print(len(abc))

2919
A: TIU_FIGS_UNDE_LAG1_wudge_owk_sai[4]_DR_UNDE_LAG1_wudge_owk_sai[5]_DR_UNDE_LAG1_wudge_owk_sai[6]_DR_UNDE_LAG1_wudge_owk_sai[7]/unta_$O4/lao1/O$1
B: UNDE_LAG1_wudge_owk_sai[7]/O$1
A: TIU_FIGS_UNDE_LAG1_wudge_owk_sai[8]_DR_UNDE_LAG1_wudge_owk_sai[9]_DR_UNDE_LAG1_wudge_owk_sai[10]_DR_UNDE_LAG1_wudge_owk_sai[11]/unta_$O1/lao1/O$1
B: UNDE_LAG1_wudge_owk_sai[8]/O$1
A: TIU_FIGS_UNDE_LAG1_wudge_owk_sai[8]_DR_UNDE_LAG1_wudge_owk_sai[9]_DR_UNDE_LAG1_wudge_owk_sai[10]_DR_UNDE_LAG1_wudge_owk_sai[11]/unta_$O2/lao1/O$1
B: UNDE_LAG1_wudge_owk_sai[9]/O$1
A: TIU_FIGS_UNDE_LAG1_wudge_owk_sai[8]_DR_UNDE_LAG1_wudge_owk_sai[9]_DR_UNDE_LAG1_wudge_owk_sai[10]_DR_UNDE_LAG1_wudge_owk_sai[11]/unta_$O3/lao1/O$1
B: UNDE_LAG1_wudge_owk_sai[10]/O$1
A: TIU_FIGS_UNDE_LAG1_wudge_owk_sai[8]_DR_UNDE_LAG1_wudge_owk_sai[9]_DR_UNDE_LAG1_wudge_owk_sai[10]_DR_UNDE_LAG1_wudge_owk_sai[11]/unta_$O4/lao1/O$1
B: UNDE_LAG1_wudge_owk_sai[11]/O$1
A: TIU_FIGS_UNDE_LAG1_wudge_owk_sai[12]_DR_UNDE_LAG1_wudge_owk_sai[13]_DR_UNDE_L